# Factory pattern

V OOP se často využívá tzv. Factory pattern - vytváříme objekty, které slouží k tvorbě jiných objektů. Vše lze abstrahovat - vytvoříme si základní Factory, ze které budou dědit konkrétní. Každá konkrétní factory pak bude vytvářet příslušné objekty. Ukažme si to na příkladu jednoduchého obchodu s nábytkem

Produkty které budeme zavádět mohou být z různých materiálů. Typ materiálu je výčtový typ.

Dále si vytvoříme trídy produktů, každá třída bude odpovídat kategorii. Objekty těchto typů slouží zejména k uchovávání informací o jednotlivých produktech / materiálech - použijeme tzv. [dataclasses anotace](https://docs.python.org/3/library/dataclasses.html), které za nás vytvoří `__init__()`, `__repr__()` a další metody

In [ ]:
from FIKEA.material import *

In [ ]:
from FIKEA.products import *

Nyní máme základní produkty, chceme jimi populovat webovou stránku - potřebujeme vytvářet listy pro jednotlivé kategorie. K tomu nám budou sloužit továrny. Nejprve musíme specifikovat, co od továrny očekáváme. Nejprve specifikujeme továrnu na materiály

In [ ]:
from FIKEA.material_factory import *

Jednotlivé produkty mohou mít stejný materiál, nechceme duplicitně vytvářet materiály. Využijeme tzv. repository pattern.

In [ ]:
from FIKEA.repositories import *


Tato repository může sloužit jak pro materiály, tak i pro produkty - je generická!

In [ ]:
from FIKEA.product_factories import *

Nyní máme vše připraveno a můžeme začít tvořit samotný program. Vybudujeme si jednoduchý webový server, který načte produkty z JSONu a vytvoří listy pro jednotlivé kategorie.

In [ ]:
from typing import cast
from FIKEA.view_controller import Response, route, get_page_menu, product_repository, not_found, bad_request, get_factories

@route("")
def home(path_args: Sequence[str], query_args: Sequence[str]) -> Response:
    if (len(path_args)) != 0:
        return not_found()
    return Response(200,
                        {"Content-type": "text/html; charset=utf-8"},
                        f"""
                        <html>
                        <head><title>FIKEA</title></head>
                        <body>
                        {get_page_menu()}
                        <div style="display: flex; flex-direction: column">
                            <h1> Fake Ikea</h1>
                            <p>
                                Ten nejlepší nábytek!
                            </p>
                        </div>
                        </body>
                        </html>
                        """)


def parse_product_reduced(_product: Product) -> str:
    return f"""
        <div style="display: flex; flex-direction: column; margin-top: 8px;">
            <div><a href="/id/{_product.id}">{_product.name}</a></div>
            <div style="display: flex; flex-direction: row">
                <div style="margin-left: 4px; margin-right: 4px;">
                    {_product.description}
                </div>
                <div style="margin-left: 4px; margin-right: 4px;">
                    {_product.get_total_price():.{2}f} Kč
                </div>
            </div>
        </div>"""


def parse_product_full(_product: Product) -> str:
    return f"""
    <div style="display: flex; flex-direction: column">
            <h3>{_product.name}</h3>
            <div style="display: flex; flex-direction: row">
                <div style="margin-left: 4px; margin-right: 4px;">
                    {_product.description}
                </div>
                <div style="margin-left: 4px; margin-right: 4px;">
                    {_product.get_total_price():.{2}f} Kč
                </div>
            </div>
            <div style="margin-left: 4px; margin-right: 4px;">
                Materiál: {_product.material}
            </div>
            {"" if not isinstance(_product, Table) else f'<div style="margin-left: 4px; margin-right: 4px;">Výška stolu: {cast(Table, _product).height}</div>'}
        </div>
    """

def single_category_products_page(category_name: str) -> Response:
    if category_name not in get_factories():
        return not_found("products", category_name)

    items = product_repository.get_all(lambda _x: isinstance(_x, get_factories()[category_name].get_created_type()))

    items_body = "".join(list(map(parse_product_reduced, items)))

    return Response(200,
                    {"Content-type": "text/html; charset=utf-8"},
                    f"""
                    <html>
                    <head><title>Produkty - {get_factories()[category_name].get_category_name()}</title></head>
                    <body>
                    {get_page_menu()}
                    {items_body}
                    </body>
                    </html>
                    """)


def all_products_page() -> Response:
    items = product_repository.get_all(lambda _x: True)

    items_body = "".join(list(map(parse_product_reduced, items)))

    return Response(200,
                    {"Content-type": "text/html; charset=utf-8"},
                    f"""
                    <html>
                    <head><title>Produkty</title></head>
                    <body>
                    {get_page_menu()}
                    {items_body}
                    </body>
                    </html>
                    """)


@route("products")
def products_page(path_args: Sequence[str], query_args: Sequence[str]) -> Response:
    if len(path_args) == 0 or path_args[0] == '':
        return all_products_page()
    if len(path_args) > 1:
        return bad_request(path_args, query_args)
    return single_category_products_page(path_args[0])


@route("id")
def single_product_page(path_args: Sequence[str], query_args: Sequence[str]) -> Response:
    if len(path_args) != 1:
        return bad_request(path_args, query_args)

    try:
        item_id = int(path_args[0])
    except ValueError:
        return bad_request(path_args, query_args)

    product = product_repository.get_by_id_or_default(item_id)

    if product is None:
        return not_found(path_args, query_args)

    product_body = parse_product_full(product)

    return Response(200,
                    {"Content-type": "text/html; charset=utf-8"},
                    f"""
                    <html>
                    <head><title>Produkty</title></head>
                    <body>
                    {get_page_menu()}
                    {product_body}
                    </body>
                    </html>
                    """
                    )

In [ ]:
from http.server import BaseHTTPRequestHandler, HTTPServer
from FIKEA.view_controller import handle_request

class IKEAServerHandler(BaseHTTPRequestHandler):

    def do_GET(self):

        response_body = handle_request(self)

        self.send_response(response_body.status_code)

        for h_n, h_v in response_body.headers.items():
            self.send_header(h_n, h_v)
        self.end_headers()

        self.wfile.write(bytes(response_body.body, "utf-8"))

A nyní web server spustíme

In [ ]:
from FIKEA.view_controller import initialize_repositories, register_factory

register_factory("chairs", ChairFactory)
register_factory("tables", TableFactory)
register_factory("beds", BedFactory)
register_factory("grates", GrateFactory)
register_factory("others", OtherProductFactory)
initialize_repositories()

webServer = HTTPServer(("localhost", 8005), IKEAServerHandler)

try:
    webServer.serve_forever()
except KeyboardInterrupt:
    pass

webServer.server_close()
print("Server stopped.")

Máme naimplementovaný jednoduchý HTTP server, který nám vypisuje naše produkty! Můžete zkusit podle typu (třídy) produktu nechat vypsat jeho specifické vlastnosti (např. šířku a hloubku pro náhledy stolů).

Vraťme se ale na teoretičtější úroveň - hierarchie typů a abstraktní třídy nám dávají poměrně velkou volnost v možnostech návrhů software, jsou s tím ale také spojené jisté problémy, jako např. takzvaný Diamond Problem

In [ ]:
class A:
    x = 10
    def a(self):
        print(f"A: {self.x}")

class B0(A):
    x = 20
    def a(self):
        print(f"B: {self.x}")

class B(B0):
    pass

class C(A):
    x = 30
    def a(self):
        print(f"C: {self.x}")

class D(B, C):
    pass

d = D()
d.a()

# Magické metody (Dunder methods)

Jako poslední si ukážeme magické metody - Python umožňuje v rámci tříd definovat metody, které pak jazyk interně používá

In [ ]:
class Vector:
    def __init__(self, *nums: float):
        self.items = nums

    def __len__(self):
        return len(self.items)

    def __add__(self, other):
        if not isinstance(other, Vector):
            return NotImplemented
        if len(self) != len(other):
            return NotImplemented
        _x = [i + j for i, j in zip(self.items, other.items)]
        return Vector(*_x)

    def __eq__(self, other):
        if not isinstance(other, Vector):
            return NotImplemented
        if len(self)!= len(other):
            return NotImplemented
        for _x, _y in zip(self.items, other.items):
            if _x != _y:
                return False
        return True

    # Porovnání podle normy vektorů
    def __le__(self, other):
        if len(self)!= len(other):
            return NotImplemented
        return sum(self.items) <= sum(other.items)

    # Porovnání podle  součtové normy vektorů
    def __lt__(self, other):
        if len(self)!= len(other):
            return NotImplemented
        return sum(self.items) < sum(other.items)

    # Porovnání podle  součtové normy vektorů
    def __ge__(self, other):
        if len(self)!= len(other):
            return NotImplemented
        return sum(self.items) >= sum(other.items)

    # Porovnání podle součtové normy vektorů
    def __gt__(self, other):
        if len(self)!= len(other):
            return NotImplemented
        return sum(self.items) > sum(other.items)

    def __str__(self):
        return f"({', '.join(list(map(str, self.items)))})"

    def __neg__(self):
        return Vector(*tuple(map(lambda _x: -_x, self.items)))

    # Součtová norma
    def __abs__(self):
        return sum(self.items)

    # "Indexátor"
    def __getitem__(self, item):
        return self.items[item]


if __name__ == "__main__":
    a = Vector(1, 2, 3)
    b = Vector(1, 1, 1)

    c = a + b
    d = -c
    print(c)
    print(d)
    print(c > a)
    print(c < a)
    print(c == c)
    print(c == a)
    print(d[1])

Pokud chceme vypsat které magické metody daný objekt implementuje, můžeme použít funkci `dir()`

In [ ]:
dir(a)

Zkusme pomocí magické metody `__call__`, která umožnuje volat na instance tříd vytvořit delegátový typ

In [ ]:
from typing import Tuple, TypeVar, Generic, Callable, Sequence

TArg = TypeVar('TArg')
TOut = TypeVar('TOut')

class Delegate(Generic[TArg, TOut]):
    def __init__(self, *function: Callable[[TArg], TOut]):
        self.function = [*function]

    def __add__(self, other):
        if callable(other):
            return Delegate(*self.function, other)


    def __sub__(self, other):
        if other in self.function:
            new_function = self.function[:]
            new_function.remove(other)
            return Delegate(*new_function)
        return NotImplemented

    def __call__(self, *args, **kwargs):
        ret = None
        for func in self.function:
            ret = func(*args, **kwargs)

        return ret

def delegate_func1(_x: int, _y: int) -> int:
    res = _x + _y
    print(res)
    return res

def delegate_func2(_x: int, _y: int) -> int:
    res = _x - _y
    print(res)
    return res

a = Delegate[Tuple[int], int](delegate_func1)

a += delegate_func2

a(1, 2)

a -= delegate_func1
print("Removed...")

x = a(1, 2)


Dalším příkladem může být kontextový manažer pomocí metod `__enter__` a `__exit__`. Vytvořme si například třídu Account, která bude simulovat transakce na účet. Pro provedení transakcí budeme používat kontext transakce, který nám umožní "rollback" v případě že transakce nelze provést.

In [ ]:
from functools import total_ordering

@total_ordering
class Account:
    def __init__(self, owner: str, amount: int = 0):
        self.owner = owner
        self.amount = amount
        self.__transaction_list = []

    def add_transaction(self, amount):
        if not isinstance(amount, int):
            raise ValueError
        if amount == 0:
            raise ValueError
        self.amount += amount
        self.__transaction_list.append(amount)

    def __len__(self):
        return len(self.__transaction_list)

    def __getitem__(self, item):
        return self.__transaction_list[item]

    def __eq__(self, other):
        if not isinstance(other, Account):
            return NotImplemented
        return self.amount == other.amount

    def __lt__(self, other):
        if not isinstance(other, Account):
            return NotImplemented
        return self.amount < other.amount

    @property
    def incomming_transactions(self) -> Sequence[int]:
        return [_x for _x in self.__transaction_list if _x > 0]

    @property
    def outgoing_transactions(self) -> Sequence[int]:
        return [_x for _x in self.__transaction_list if _x < 0]

    @property
    def transactions(self) -> Sequence[int]:
        return self.__transaction_list[:]

    def __enter__(self):
        self.__transaction_list_copy = self.__transaction_list[:]
        self.__amount_copy = self.amount
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type:
            self.__transaction_list = self.__transaction_list_copy[:]
            self.amount = self.__amount_copy
            del self.__transaction_list_copy
            del self.__amount_copy
            print ("Rolling back transactions...")
            return
        print("Transactions OK")



acc = Account("Alice", 20)
try:
    with acc as a:
        a.add_transaction(-21)
        if a.amount < 0:
            raise ValueError("Debt detected!")
except ValueError as e:
    print(e)

print(a.transactions)
print(a.amount)